In [2]:
# import libraries
%matplotlib inline
import numpy as np
#import csv
import matplotlib.pyplot as plt
import pandas as pd
import glob
import ulmo
import os
import scipy.spatial

In [3]:
ghcn = pd.read_fwf('data/ghcnd-stations.txt', colspecs = [(0,10), (12,19), (21,29), (31,36),(38,40), (41,70), (72,74),(76,78),(80,85)], header = None) 
colnames = ['GHCN ID', 'lat', 'lon', 'elevation', 'state', 'name', 'gsn flag', 'HCN/CRN FLAG', 'WMO ID']
ghcn.columns = colnames

# from http://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt
# FORMAT OF "ghcnd-stations.txt"
#
# ------------------------------
# Variable   Columns   Type
# ------------------------------
# ID            1-11   Character
# LATITUDE     13-20   Real
# LONGITUDE    22-30   Real
# ELEVATION    32-37   Real
# STATE        39-40   Character
# NAME         42-71   Character
# GSN FLAG     73-75   Character
# HCN/CRN FLAG 77-79   Character
# WMO ID       81-85   Character
# ------------------------------

# These variables have the following definitions:

# ID         is the station identification code.  Note that the first two
#            characters denote the FIPS  country code, the third character 
#            is a network code that identifies the station numbering system 
#            used, and the remaining eight characters contain the actual 
#            station ID. 

#            See "ghcnd-countries.txt" for a complete list of country codes.
# 	   See "ghcnd-states.txt" for a list of state/province/territory codes.

#            The network code  has the following five values:

#            0 = unspecified (station identified by up to eight 
# 	       alphanumeric characters)
# 	   1 = Community Collaborative Rain, Hail,and Snow (CoCoRaHS)
# 	       based identification number.  To ensure consistency with
# 	       with GHCN Daily, all numbers in the original CoCoRaHS IDs
# 	       have been left-filled to make them all four digits long. 
# 	       In addition, the characters "-" and "_" have been removed 
# 	       to ensure that the IDs do not exceed 11 characters when 
# 	       preceded by "US1". For example, the CoCoRaHS ID 
# 	       "AZ-MR-156" becomes "US1AZMR0156" in GHCN-Daily
#            C = U.S. Cooperative Network identification number (last six 
#                characters of the GHCN-Daily ID)
# 	   E = Identification number used in the ECA&D non-blended
# 	       dataset
# 	   M = World Meteorological Organization ID (last five
# 	       characters of the GHCN-Daily ID)
# 	   N = Identification number used in data supplied by a 
# 	       National Meteorological or Hydrological Center
# 	   R = U.S. Interagency Remote Automatic Weather Station (RAWS)
# 	       identifier
# 	   S = U.S. Natural Resources Conservation Service SNOwpack
# 	       TELemtry (SNOTEL) station identifier
#            W = WBAN identification number (last five characters of the 
#                GHCN-Daily ID)

# LATITUDE   is latitude of the station (in decimal degrees).

# LONGITUDE  is the longitude of the station (in decimal degrees).

# ELEVATION  is the elevation of the station (in meters, missing = -999.9).


# STATE      is the U.S. postal code for the state (for U.S. stations only).

# NAME       is the name of the station.

# GSN FLAG   is a flag that indicates whether the station is part of the GCOS
#            Surface Network (GSN). The flag is assigned by cross-referencing 
#            the number in the WMOID field with the official list of GSN 
#            stations. There are two possible values:

#            Blank = non-GSN station or WMO Station number not available
#            GSN   = GSN station 

# HCN/      is a flag that indicates whether the station is part of the U.S.
# CRN FLAG  Historical Climatology Network (HCN).  There are three possible 
#           values:

#            Blank = Not a member of the U.S. Historical Climatology 
# 	           or U.S. Climate Reference Networks
#            HCN   = U.S. Historical Climatology Network station
# 	   CRN   = U.S. Climate Reference Network or U.S. Regional Climate 
# 	           Network Station

# WMO ID     is the World Meteorological Organization (WMO) number for the
#            station.  If the station has no WMO number (or one has not yet 
# 	   been matched to this station), then the field is blank.

# --------------------------------------------------------------------------------
# --------------------------------------------------------------------------------
giss = pd.read_fwf('data/v3.temperature.inv.txt',skiprows = 39, header = None,
                  colspecs=[(0,3),(3,8),(8,11), (12,44),(44,49), (52,58), (58,63), (63,67), (67,68), (69,73), (73,75), (75, 77), (78,79), (79,81), (81,82),(82,84), (84,100), (100,102), (103,106)])
colnames = ['icc country code', 'WMO ID', '3 digit modifier', 'name','lat', 'lon', 'elevation', 'TELe', 'P', 'Pop', 'Tp', 'V', 'Lo', 'Co', 'Airport', 'ds', 'Vege', 'bi', 'BI']
giss.columns = colnames

# LEGEND  
# ======
# icc  =3 digit country code; the first digit represents WMO region/continent
# WMO_#=5 digit WMO station number
# ...  =3 digit modifier; 000 means the station is probably the WMO
#       station; 001, etc. mean the station is near that WMO station
# Name =30 character station name
# Lat  =latitude in degrees, negative = South of Equator
# Lon  =longitude in degrees, negative = West of Greenwich (England)
# Elev =station elevation in meters, missing is -999
# TEle =station elevation interpolated from TerrainBase gridded data set
# P    =R if rural (not associated with a town of >10,000 population)
#       S if associated with a small town (10,000-50,000 population)
#       U if associated with an urban area (>50,000 population)
# Pop  =population of the small town or urban area in 1000s
#       If rural, no analysis:  -9.
# Tp   =general topography around the station:  FL flat; HI hilly,
#       MT mountain top; MV mountainous valley or at least not on the top
#       of a mountain.
# V    =general vegetation near the station based on Operational
#       Navigation Charts;  MA marsh; FO forested; IC ice; DE desert;
#       CL clear or open;  xx information not provided
# Lo   =CO if station is within 30 km from the coast
#       LA if station is next to a large (> 25 km**2) lake
#       no if neither of the above
#       Note: Stations which are both CO and LA will be marked CO
# Co   =distance in km to the coast if Lo=CO, else -9
# A    =A if the station is at an airport; else x
# ds   =distance in km from the airport to its associated
#       small town or urban center (not relevant for rural airports
#       or non airport stations in which case ds=-9)
# Vege =gridded vegetation for the 0.5x0.5 degree grid point closest
#       to the station from a gridded vegetation data base. 16 characters.
# bi   =brightness index    A=dark B=dim C=bright   (comment added by R.Ruedy)
# BI   =brightness index    0=dark -> 256 =bright   (based on satellite night light data)

see: http://stackoverflow.com/questions/35296935/python-calculate-lots-of-distances-quickly

In [201]:
# compute distances between all stations
tree = scipy.spatial.cKDTree(giss[['lon', 'lat']].values, leafsize=100)

# query the closest point 
closestInd = tree.query(giss[['lon', 'lat']].values[11,:], k =2, distance_upper_bound=6)[1][1]

In [5]:
# find the synoptic station and pair it with the closest stations
city = 'MADISON'
synopticID = ghcn[ghcn['name'].str.contains(city) & ~np.isnan(ghcn['WMO ID'])]['WMO ID'].iloc[0]
closeststations = tree.query(giss[giss['WMO ID']==synopticID][['lon', 'lat']].values , k =5, distance_upper_bound=5)

In [6]:
# find the synoptic station and pair it with the closest stations
#city = 'MADISON'
frames = []
selectedCities = ['MADISON', 'NEW YORK', 'BIRMINGHAM', 'MINNEAPOLIS', 'HOUSTON', 'ATLANTA', 'SEATTLE', 'LOS ANGELES', # US cities
                  'MEXICO CITY', 'MONTERREY', 'MERIDA', 'PUEBLA', # MEXICO
                  'PARIS', 'LYON', 'NICE', 'BORDEAUX', 'MARSEILLE', # French cities
                  'LONDON', 'BIRMINGHAM', 'MANCHESTER', 'YORK', #UK
                   'MUNICH', 'BERLIN', 'FRANKFURT', 'HAMBURG', 'BREMEN', 'HANOVER',#eu
                  'MOSCOW', 'VLADIVOSTOK', 'ST PETERSBURG', 'SAMARA', #russia
                  'CASABLANCA', 'RABAT', 'TANGIER', # morocco 
                 'CAIRO', 'DUBAI', 'AMMAN', 'BEIRUT','BAALBEK', 'JERUSALEM', 'TEL AVIV', 'JEDDAH', # Other arab cities]
                'NAIROBI', 'DAR ES SALAAM', 'OUGADOUGOU', 'CAPETOWN', 'PRETORIA', 'MAPUTO', 'WINDHOEK', 'LUANDA',# sub-saharan
                  'DELHI', 'TASHKENT', 'KARACHI', 'LAHORE', 'KABUL', 'TEHRAN',# Central Asia
                  'HANOI', 'PHNOM PENH', 'BANGKOK', 'XIAN', 'ZHENDOU', 'SHANGHAI', 'SHENYANG', 'SEOUL', 'INCHEON', 'TOKYO','KYOTO', 'OSAKA',] 
                  
for city in selectedCities: #['LONDON', 'NEW YORK','MEXICO CITY', 'PARIS', 'MADISON']:
    # first check that there is a WMO station 
    if ghcn[ghcn['name'].str.contains(city)].values.shape[0] ==0: 
        print 'No station found for %s'%city
    else : 
        synopticID = ghcn[ghcn['name'].str.contains(city) & ~np.isnan(ghcn['WMO ID'])]['WMO ID']
        if synopticID.shape[0] == 0 :
            print 'No synoptic station found for %s'%city
        else: 
            synopticID = synopticID.iloc[0].astype(int)
            closeststations = tree.query(giss[giss['WMO ID']==synopticID][['lon', 'lat']].values , k =5, distance_upper_bound=5)
            #print city, synopticID

            # Check that there is an entry for the city
            if giss[giss['WMO ID'] == synopticID].values.shape[0]==0:
                print 'No GISS data found for synoptic station found for %s, %s'%(city, synopticID)
            else: 
                #print 'Station found for %s'%city
                # Now classify the synoptic station as urban/rural and pair it with the appropriate station
                synopticBrightness = giss[giss['WMO ID'] == synopticID]['BI'].iloc[0]

                if synopticBrightness > 20 :
                # if the synoptic station is urban, then search for rural station

                    urbanID = synopticID
                    ruralID = giss.iloc[giss.iloc[closeststations[1][0]]['BI'].argmin()]['WMO ID']
                    frames.append([city,urbanID, ruralID])
                else : 
                # if the synoptic station is rural, then search for a more urban station

                    ruralID = synopticBrightness
                    urbanID = giss.iloc[giss.iloc[closeststations[1][0]]['BI'].argmax()]['WMO ID']
                    frames.append([city, urbanID, ruralID])

No GISS data found for synoptic station found for HOUSTON, 72244
No station found for PUEBLA
No GISS data found for synoptic station found for NICE, 71712
No synoptic station found for MUNICH
No station found for FRANKFURT
No synoptic station found for HANOVER
No synoptic station found for ST PETERSBURG
No station found for CASABLANCA
No GISS data found for synoptic station found for RABAT, 38049
No synoptic station found for CAIRO
No GISS data found for synoptic station found for DUBAI, 41194
No synoptic station found for AMMAN
No station found for BEIRUT
No station found for BAALBEK
No synoptic station found for JERUSALEM
No synoptic station found for TEL AVIV
No GISS data found for synoptic station found for JEDDAH, 41024
No station found for OUGADOUGOU
No station found for CAPETOWN
No GISS data found for synoptic station found for MAPUTO, 67331
No GISS data found for synoptic station found for DELHI, 71573
No station found for KARACHI
No station found for HANOI
No station found for

In [227]:
city = 'NEW YORK'
# city = 'LONDON'
# country = 425#651 #425 US, 651 UK
cities = 
countries = 425
frames = []
for (city, country) in zip(cities,countries): 
    ids = giss[(giss['name'].str.contains(city)) & (giss['icc country code']==country)]
    if ids.shape[0]==0 : 
        print 'no stations found for %s'%city
    elif ids.shape[0]==1 :
        id = ids['WMO ID'].iloc[0]
    else: 
        print 'picking the most synoptic station'
        id = ids.loc[ids['3 digit modifier'].argmin()]['WMO ID']
        #id = 
    # find closest couple stations
    closeststations = tree.query(giss[giss['WMO ID']==id ].iloc[0][['lon', 'lat']].values, k =5, distance_upper_bound=.5)

    urbanID = giss.loc[giss.iloc[closeststations[1]]['BI'].argmax()]['WMO ID']
    ruralID = giss.loc[giss.iloc[closeststations[1]]['BI'].argmin()]['WMO ID']
    #print closeststations[1]
    #print city, id 
    frames.append([city, urbanID, ruralID])

picking the most synoptic station


In [15]:
atlas = pd.read_csv('data/sampleAtlas.csv')

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province
0,Melbourne,Melbourne,-37.820031,144.975016,2131812.5,Australia,AU,AUS,Victoria
1,Sydney,Sydney,-33.920011,151.185180,4135711.0,Australia,AU,AUS,New South Wales
2,London,London,51.499995,-0.116722,7994104.5,United Kingdom,GB,GBR,Westminster
3,Birmingham,Birmingham,52.474974,-1.919997,1634666.5,United Kingdom,GB,GBR,West Midlands
4,Philadelphia,Philadelphia,39.999973,-75.169996,3504775.0,United States of America,US,USA,Pennsylvania
5,Detroit,Detroit,42.329960,-83.080056,2526135.0,United States of America,US,USA,Michigan
6,Anchorage,Anchorage,61.219970,-149.900215,252068.0,United States of America,US,USA,Alaska
7,San Francisco,San Francisco,37.740008,-122.459978,2091036.0,United States of America,US,USA,California
8,Denver,Denver,39.739188,-104.984016,1930799.5,United States of America,US,USA,Colorado
9,Houston,Houston,29.819974,-95.339979,4053287.0,United States of America,US,USA,Texas


In [24]:
atlas = pd.read_csv('data/sampleAtlas.csv') # derived  from http://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-populated-places/
tree = scipy.spatial.cKDTree(ghcn[['lon', 'lat']].values, leafsize=100)
import sys
sys.path.append('../cityheat/Bmore/2015/')
import spatialfunctions

In [159]:
frames = []
rasterfile = 'data/F182013.v4/F182013.v4c_web.stable_lights.avg_vis.tif'

for i in range(0, atlas.shape[0]): 
    lat = atlas.iloc[i]['lat']
    lon = atlas.iloc[i]['lng']
    city = atlas.iloc[i]['city']
    closeststations = tree.query([lon,lat], k =5, distance_upper_bound=.25)
    lons = ghcn.iloc[closeststations[1]]['lon'].values
    lats = ghcn.iloc[closeststations[1]]['lat'].values
    BIs = spatialfunctions.extract_raster_values(lons, lats, rasterfile)
    frames.append([city, BIs.max(), BIs.min()])

IndexError: positional indexers are out-of-bounds

In [79]:
rasterfile = 'data/F182013.v4/F182013.v4c_web.stable_lights.avg_vis.tif'

X = ghcn['lon'].values
Y = ghcn['lat'].values 
DN = spatialfunctions.extract_raster_values(X[15000:16000], Y[15000:16000],rasterfile)

In [96]:
allNightLights = np.zeros(ghcn.shape[0])
rasterfile = 'data/F182013.v4/F182013.v4c_web.stable_lights.avg_vis.tif'

X = ghcn['lon'].values
Y = ghcn['lat'].values 

for i in [0,1,2,3,4,5,6,7,8,9,10]:
    ind = i*10000
    DN = spatialfunctions.extract_raster_values(X[ind:ind+5999], Y[ind:ind+5999],rasterfile)
    allNightLights[ind:ind+9999] = DN

AttributeError: 'NoneType' object has no attribute 'shape'

In [155]:
from osgeo import ogr, osr
import os
from shapely.geometry import Point
import shapely.geometry
import shapely.wkt
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
import gdal

rasterfile = 'data/F182013.v4/F182013.v4c_web.stable_lights.avg_vis.tif'
layer = gdal.Open(rasterfile)
gt =layer.GetGeoTransform()
bands = layer.RasterCount
src = layer.GetRasterBand(1)

In [144]:
gt

(-180.00416666665, 0.0083333333, 0.0, 75.00416666665, 0.0, -0.0083333333)

In [157]:
print lon,lat

-0.116721844 51.49999473


In [206]:
#rasterfile = 'data/F182013.v4/F182013.v4c_web.stable_lights.avg_vis.tif'
rasterfile = 'data/F182013.v4/F182013.v4c_web.avg_vis.tif'
layer = gdal.Open(rasterfile)
gt =layer.GetGeoTransform()
bands = layer.RasterCount
src = layer.GetRasterBand(1)

BI = np.zeros(X.shape)
i = 0 
for x,y in zip(X,Y): 
    rasterx = int((x - gt[0]) / gt[1])
    rastery = int((y - gt[3]) / gt[5])
    BI[i] = src.ReadAsArray(rasterx,rastery, win_xsize=1, win_ysize=1)
    i = i+1

In [214]:
np.save('brightnessGHCN.npy', BI)